# ETL reviews Yelp

### Optimización de los datasets originales

El archivo es muy grande para la tranformacion directa del mismo. Por ende lo dividimos en partes para que seas mas optimo y mas facil de trabajarlo. Lo particionamos sera de 300000 filas, que nos da como resultado 24 archivos.

In [1]:
# # Generamos partes de un tamaño de chunck 300000, genera 24 partes en json.
# def dividir_json_en_partes(archivo_entrada, tamaño_maximo, prefijo_salida):
#     with open(archivo_entrada, 'rb') as f_entrada:
#         # Inicializa la lista para almacenar cada parte del JSON
#         partes = []
#         numero_parte = 1

#         # Itera sobre cada línea del archivo JSON
#         for linea_binaria in f_entrada:
#             # Decodifica la línea binaria como UTF-8
#             linea = linea_binaria.decode('utf-8')

#             # Decodifica la línea como JSON
#             dato = json.loads(linea)

#             # Agrega el dato a la parte actual
#             partes.append(dato)

#             # Si la parte alcanza el tamaño máximo, guárdala y reinicia la lista
#             if len(partes) >= tamaño_maximo:
#                 guardar_parte(prefijo_salida, numero_parte, partes)
#                 numero_parte += 1
#                 partes = []

#         # Si hay datos restantes, guárdalos como la última parte
#         if partes:
#             guardar_parte(prefijo_salida, numero_parte, partes)

# def guardar_parte(prefijo_salida, numero_parte, datos):
#     nombre_salida = f"{prefijo_salida}_{numero_parte}.json"
#     with open(f'../extras/Datasets/Yelp/Pruebas/reviews_{nombre_salida}', 'w') as f_salida:
#         json.dump(datos, f_salida, indent=2)  # Puedes ajustar el nivel de indentación según tus preferencias

# # Especifica el archivo JSON de entrada, el tamaño máximo por parte y el prefijo para los archivos de salida

# # Llama a la función para dividir el JSON en partes
# dividir_json_en_partes('../extras/Datasets/Yelp/review.json', 300000, 'parte') 

Una vez dividido en partes lo guardamos en tipo parquet comprimido con gzip para que reduzca su tamaño.

In [2]:
# cantidad = list(range(1,25))

# def convertir_json_a_parquet(archivo_json, archivo_parquet):
#     # Lee el archivo JSON en un DataFrame de pandas
#     with open(archivo_json, 'r') as f:
#         datos_json = json.load(f)

#     # Convierte el JSON a un DataFrame de pandas
#     dataframe = pd.DataFrame(datos_json)

#     # Convierte el DataFrame de pandas a una tabla de PyArrow
#     tabla = pa.Table.from_pandas(dataframe)

#     # Escribe la tabla en un archivo Parquet
#     pq.write_table(tabla, archivo_parquet, compression='gzip')

# for i in cantidad:
#     # Especifica el archivo JSON de entrada y el archivo Parquet de salida
#     archivo_json = f'Datasets/Yelp/Pruebas/reviews_parte_{i}.json'
#     archivo_parquet = f'Datasets/Yelp/reviewsParquet/reviews_parte_{i}.gz.parquet'

#     # Llama a la función para convertir el JSON a Parquet
#     convertir_json_a_parquet(archivo_json, archivo_parquet)

### Tranformacion de los datos

#### Analisis de sentimiento

Importamos las librerias necesarias, y generamos el modelo de NLP para el analisis de sentimiento de la review.

In [1]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download(['vader_lexicon', 'stopwords', 'punkt', 'names'])

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Damian\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Damian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Damian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Damian\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


Para saber si un comentario es positivo, negativo o neutro, utilizamos polarity_scores que devuelve un diccionario de 4 claves, neg, neu, pos y compound. Las que nos interesa para clasificar la reseña es el compound, que es una puntuacion compuesta de todo el texto. 

Para calificarlo ademas de usar esta funcion, utilizamos la clasificacion que da el resto de los usuarios, tomamos como esto a las estrellas que tiene la reseña. Por ende este valor lo dividimos por 5 asi nos quedan valores mas chicos y lo sumamos con el valor del compound previamente calculado.

Generamos esta funcion puntajeNLP para poder redondear este valor calculado por la suma antes mencionada, y ahi redondearlo a 2 (positivo), 1 (negativo), 0(neutro).

In [4]:
def puntajeNLP(x):
    if x > 1.5:
        return 2 # Positivo
    elif x >= 1:
        return 0 # Neutro
    else: 
        return 1 # Negativo

In [ ]:
# Hacemos la tranformacion de todas las partes realizadas anteriormente.

for i in range(1,25):
    # Leemos el archivo
    df = pd.read_parquet(f'../extras/Datasets/Yelp/reviewsParquet/reviews_parte_{i}.gz.parquet')
    # Nos quedamos con las columnas que necesitamos
    df = df[['review_id', 'user_id', 'business_id','stars', 'text', 'date']]

    # Realizamos el analisis de sentimiento, la division de las estrellas y la suma de ambas para la clasificacion de la reseña.
    analisis = df['text'].apply(lambda x: sid.polarity_scores(x)["compound"])
    valorEstrellas = df['stars'] / 5 
    analisis += valorEstrellas
    analisis = analisis.apply(lambda x: puntajeNLP(x))

    # Reemplazamos la reseña por el analisis de sentimiento ya que son los datos necesarios y además para reducir el peso del archivo.
    df['text'] = analisis

    # Eliminamos stars ya que no es necesario
    df.drop(columns='stars', inplace=True)

    # Renombramos las columnas
    df.columns = ['review_id', 'user_id', 'business_id','sentiment', 'date']

    # Cambiamos tipo de dato.
    df['sentiment'] = df['sentiment'].astype('int8')

    # Exportamos el archivo para su proxima union.
    df.to_parquet(f'../extras/Datasets optimizados/Yelp/reviews_parte_{i}_gz.parquet', compression='gzip')

Unimos todos los datasets

In [7]:
df = pd.read_parquet('../extras/Datasets optimizados/Yelp/reviews procecados particionado/reviews_parte_1_gz.parquet')
for i in range(2,25):
    # Leemos el archivo
    aux = pd.read_parquet(f'../extras/Datasets optimizados/Yelp/reviews procecados particionado/reviews_parte_{i}_gz.parquet')
    df = pd.concat([df,aux], axis=0, ignore_index=True)

df.to_parquet('../extras/Datasets optimizados/yelp_total_reviews.gz.parquet', compression='gzip')

Hacemos el merge con los negocios que vamos a utilizar segun el criterio.

In [14]:
reviewsDf = pd.read_parquet('../extras/Datasets optimizados/Yelp/yelp_total_reviews.gz.parquet')

# Dataset de negocios ya filtrado por estado y tipo de negocio.
negociosDf = pd.read_json('../datasets/processed/yelp/bussiness_yelp.json.gz')

Generamos el dataset de las reviews que vamos a utilizar, y lo guardamos.

In [15]:
reviewsDf = reviewsDf.merge(negociosDf['business_id'], how='inner')
reviewsDf.to_parquet('../datasets/processed/yelp/reviews_yelp.parquet.gz', compression='gzip')